In [2]:
!git clone https://github.com/Anonymous-GT11/SAFRAN_ENSAI_2025

Cloning into 'SAFRAN_ENSAI_2025'...
remote: Enumerating objects: 68, done.
remote: Counting objects: 100% (68/68), done.
remote: Compressing objects: 100% (63/63), done.
remote: Total 68 (delta 9), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (68/68), 1.09 MiB | 12.02 MiB/s, done.
Resolving deltas: 100% (9/9), done.


In [ ]:
import numpy as np

from performancemodel.turboreactors import SimpleCorpsSimpleFlux_Design, SimpleCorpsSimpleFlux_degrad_n_ratio_beta

from performancemodel.base_system import h_SCSF

from performancemodel.scenarios import Scenario




%reload_ext autoreload
%autoreload 2

# Design Turboreactor

We first present a simple turboreactor with all equations and values set by design. The Simple Corps Simple Flux turboreactor has 3 modules that have an impact on the temperature and pressure of the air coming through the engine:
1. A compressor
2. Combustion chamber
3. HP Turbine

We also have a fan and a nozzle but we consider that they do not degrade and do not change from one model to another.

We define several plan in the engine, each plan is at the entry or at the exit of a module. At plan 0, 1 and 2, the air goes by without modification in its pressure or temperature.
At plan 3, the air was compressed, at plan 4 the air was heated by the combustion chamber and the HP Turbine expanded the air. A plot of the turboreactor can be found in the plot folder of this repo.

Every constant or parameters of the engine is fixed inside the methods of the class.



We initialize the Turboreactor by design with the class SimpleCorpsSimpleFLux_Design()

In [ ]:

turbo_design = SimpleCorpsSimpleFlux_Design()


Then we can compute a pass forward through the engine.

In [ ]:
turbo_design.pass_forward(verbose=1)

P2 191801.04700911307
T2 360.0
W4 14.807195600842178
W2R 8.743400872860022
T3 749.5943981371744
T2 360.0
T5 1310.4056018628257
T5is 1267.1173354031396
P4 1822109.946586574
P5 651421.7694786084
P8 651421.7694786084
T8 1310.4056018628257
Wf 0.32846050647756403


It is also possible to compute plan by plan if we respect the order of the modules. Otherwise there will be errors.

In [ ]:
turbo_design.compute_plan_0()
turbo_design.compute_plan_1()
turbo_design.compute_plan_2()
turbo_design.compute_plan_3()

Then, we can output and print any variables computed inside the engine.

In [ ]:
print(turbo_design.T4)
print(turbo_design.W5)

1700
14.807195600842178


# Turboreactor

The next class of turboreactor have a few more features and can be configured to some extent. There are several differences with the design turboreactor.
- The compressor parameters are not fixed, the pressure ratio between entering and exiting air pressure and the efficiency of the compressor is computed thanks to a map.
- It is possible to choose a degradation coefficient for the compressor and the turbine. The degradation of the combustion chamber are too small to be considered in this simulator.
    - For the compressor, the degradation coefficient will modify the map between air mass flow and pressure ratio / efficiency
    - For the turbine, the degradation coefficient will modify the temperature of the air at the exit of the turbine.
- We need to specify a power engine parameter N between 0.7 and 1.1 and a degradation coefficient for the compressor.
- We need to specify the constant reduced air mass flow at the exit of the compressor W3R. A value between 3 and 5 is a good value because it is in the range of observed value when the system was run with a Wf between 0.5 and 10 for any N.

- Since certain parameters are not fixed, we do not know the air mass flow at the entry of the engine, the pressure ratio of the compressor and the operating line of the engine. To calculate these values, the system iterates over the value of a variable named beta which is the operating line of until the system reach an equilibrium and on the fuel flow inside the combustion chamber. That is to say, the reduced air mass flow at the exit of the engine is the same observed at the operating point of our engine, same for the pressure ratio.




In [ ]:
turboreactor = SimpleCorpsSimpleFlux_degrad_n_ratio_beta(N=0.9, degrad_comp=1, degrad_turb=1, W3R_design=4, ratio_design=1100)



The next line run the simulator and outputs the requested measurements. Requested measurements are any number of the following attributes :

- T0, T1, T2, T3, T4, T5, T8 which are the temperature at each plan of the engine (in K)
- P0, P1, P2, P3, P4, P5, P8 which are the pressure at each plan of the engine (in bar)
- W0, W1, W2, W3, W4, W5, W8 which are the airflow at each plan of the engine (in kg/s)
- W2R, W3R, W4R, W5R, W8R which are the corrected airflow at each plan of the engine where it is computed (plan 2 through 8) in (kg/s)
- Wf which is the fuel flow in the combustion chamber (in kg/s)
- M8 which is the mach of the air at the exit of the nozzle (in % of M1)
- F which is the thrust of the engine (in Newton)

default is ["P3", "T4", "W5R"].

As a reminder, a turboreactor and the position of each plan is available in the plot folder

In [ ]:
measurements = turboreactor.run()

In [ ]:
measurements


{'P3': 1901097.943519581, 'T4': 1898.4757339388448, 'W5R': 18.92011385522376}

We can also access any of the values once we either used turboreactor.run() or turboreactor.simulator() and then call turboreactor.captor

In [ ]:
print(turboreactor.T0)
print(turboreactor.T5)
print(turboreactor.W3R)

360.0
1479.1444665616382
4


We can then play with the different parameters such as power engine (N), and the degradation coefficient degrad_comp and degrad_comb.
We recommand to not change too much the W4R parameter as the reduced air flow exiting the combustion chamber as it could lead to out of definition context. But it is still possible to change it to observe how it affects the turboreactor outputs.

In [ ]:
turboreactor_lowN = SimpleCorpsSimpleFlux_degrad_n_ratio_beta(N=0.7)
turboreactor_degradedcomp = SimpleCorpsSimpleFlux_degrad_n_ratio_beta(N=0.9, degrad_comp=0.98)
turboreactor_degradedturbine = SimpleCorpsSimpleFlux_degrad_n_ratio_beta(N=0.9, degrad_comp=0.98, degrad_turb=0.98)

### With measurements function

We also created a function in the base_system.py called h_SCSF which can be used to directly initialize a turboreactor and compute the measurements.

Here one can customize the engine thanks to parameters :
- N (engine power) defaults to 0.9 but can vary between 0.7 and 1.1
- W3R (corrected airflow compressor exits) defaults to 3 but can vary between 2 and 8
- ratio (ratio between corrected airflow nozzle exits and design engine) defaults to 1100 but can vary between 800 and 1500

One can also customize the sensors from which we want measurements as in the previous step. Default to P3, T4 and W5R

In [ ]:
S = ["P3", "T5", "F", "Wf"]

In [ ]:
measurements = h_SCSF([[1,1], [0.99, 0.99], [0.98,0.98]], S=S)

In [ ]:
measurements

array([[1.90110390e+06, 1.47916499e+03, 4.18657661e+01, 1.49434783e+00],
       [1.90261306e+06, 1.51408029e+03, 4.10181904e+01, 1.52675788e+00],
       [1.90448014e+06, 1.55057089e+03, 4.01772825e+01, 1.56042149e+00]])

In [ ]:
measurements_lowN = h_SCSF([[1,1], [0.99, 0.99], [0.98,0.98]], S=S, N=0.7)

In [ ]:
measurements_lowN

array([[6.04065889e+05, 8.14522873e+02, 1.52541322e+01, 2.39168349e-01],
       [5.98025231e+05, 8.24723369e+02, 1.49222105e+01, 2.42147446e-01],
       [5.91984572e+05, 8.35148891e+02, 1.45938363e+01, 2.45137916e-01]])

# Degradation Scenario

In the following, we introduce the degradation scenario available at the moment. More scenarios will be added as they are implemented.

As for the turboreactor, we created a class Scenario. One must specify the speed of the degradation, it can be fast or slow, and which health indicators are concerned, it can be the combustion chamber, the compressor or both. The speed of the degradation decides the drop in values at which the parameters degrades at each step. The values of the drop can be change directly in the scenarios.py files. At the moment, the degradation are completely deterministic but randomization will be added in the future.

In [ ]:
slow_scenario = Scenario("slow", "both")
list_degradation_compressor, list_degradation_turbine = slow_scenario.generate_degradation(20,1,1)

In [ ]:
print(list_degradation_compressor)

[1.0, 0.99995, 0.9999, 0.99985, 0.9998, 0.99975, 0.9997, 0.99965, 0.9996, 0.99955, 0.9995, 0.99945, 0.9994, 0.99935, 0.9993, 0.99925, 0.9992, 0.99915, 0.9991, 0.99905]
